# The diet problem in PuLP
Source: https://github.com/keyush06/Linear-Optimization-Problem-using-PuLP

## Problem

___Suppose you are a well-established dietician and are responsible to advise one of your customers on the best possible food plan he should follow in order to attain "optimum nutrition". However, there are some restrictions in terms of the budget and the variety of food that needs to be included in the diet plan. The data set used can be downloaded from here and the jupyter notebook could be downloaded from my Github repository.___

## Intuition

__Minimize the cost of the best food plan(inclusive of food items), given some constraints (on total calories but also on each of the nutritional components e.g. fat, vitamin C, iron, etc.)__

__The Cost function is the total cost of the food items which we are trying to minimize. Why? Because the cost should be minimal, at the same time, the nutritional value derived from the combination of different food items should be maximum, considering the maximum and minimum constraints given in the data. The inequality constraints are given by the minimum and maximum bounds on each of the nutritional components.__

## Import everything from the PuLP library and other libraries

In [1]:
from pulp import *
import numpy as np, pandas as pd
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
import pulp

## Formulate the Optimization Problem

In [3]:
prob = LpProblem('Diet_Problem', LpMinimize)

### Reading the data set and creating some dictionaries

In [4]:
#Now create some dictionaries in order to extract information from the diet table.

#Reading the data
df = pd.read_excel('diet.xlsx',nrows=17)
df.head()

,Foods,Price/Serving,Serving Size,Calories,Cholesterol (mg),Total_Fat (g),Sodium (mg),Carbohydrates (g),Dietary_Fiber (g),Protein (g),Vit_A (IU),Vit_C (IU),Calcium (mg),Iron (mg)
0,Frozen Broccoli,0.48,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,Frozen Corn,0.54,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
2,Raw Lettuce Iceberg,0.06,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1
3,Baked Potatoes,0.18,1/2 Cup,171.5,0.0,0.2,15.2,39.9,3.2,3.7,0.0,15.6,22.7,4.3
4,Tofu,0.93,1/4 block,88.2,0.0,5.5,8.1,2.2,1.4,9.4,98.6,0.1,121.8,6.2


In [5]:
#List of food items
food = list(df.Foods)

#The list of items
count=pd.Series(range(1,len(food)+1))
print('List of different food items is here follows: -')
food_s = pd.Series(food)

#Convert to data frame
f_frame = pd.concat([count,food_s],axis=1,keys=['S.No','Food Items'])
f_frame

List of different food items is here follows: -


,S.No,Food Items
0,1,Frozen Broccoli
1,2,Frozen Corn
2,3,Raw Lettuce Iceberg
3,4,Baked Potatoes
4,5,Tofu
5,6,Roasted Chicken
6,7,Spaghetti W/ Sauce
7,8,Raw Apple
8,9,Banana
9,10,Wheat Bread


In [6]:
# Create a dictinary of costs for all food items
costs = dict(zip(food,df['Price/Serving']))

#Create a dictionary of calories for all items of food
calories = dict(zip(food,df['Calories']))

#Create a dictionary of cholesterol for all items of food
chol = dict(zip(food,df['Cholesterol (mg)']))

#Create a dictionary of total fat for all items of food
fat = dict(zip(food,df['Total_Fat (g)']))

#Create a dictionary of sodium for all items of food
sodium = dict(zip(food,df['Sodium (mg)']))

#Create a dictionary of carbohydrates for all items of food
carbs = dict(zip(food,df['Carbohydrates (g)']))

#Create a dictionary of dietary fiber for all items of food
fiber = dict(zip(food,df['Dietary_Fiber (g)']))

#Create a dictionary of protein for all food items
protein = dict(zip(food,df['Protein (g)']))

#Create a dictionary of vitamin A for all food items
vit_A = dict(zip(food,df['Vit_A (IU)']))

#Create a dictionary of vitamin C for all food items
vit_C = dict(zip(food,df['Vit_C (IU)']))

#Create a dictionary of calcium for all food items
calcium = dict(zip(food,df['Calcium (mg)']))

#Create a dictionary of iron for all food items
iron = dict(zip(food,df['Iron (mg)']))


In [7]:
#We just run one of the dictionaries to see how these look like
iron

{'Frozen Broccoli': 2.3,
 'Frozen Corn': 0.3,
 'Raw Lettuce Iceberg': 0.1,
 ' Baked Potatoes': 4.3,
 'Tofu': 6.2,
 'Roasted Chicken': 1.8,
 'Spaghetti W/ Sauce': 2.3,
 'Raw Apple': 0.2,
 'Banana': 0.4,
 'Wheat Bread': 0.7,
 'White Bread': 0.8,
 'Oatmeal Cookies': 0.5,
 'Apple Pie': 0.1,
 'Scrambled Eggs': 0.7,
 'Turkey Bologna': 0.4,
 'Beef Frankfurter': 0.6,
 'Chocolate Chip Cookies': 0.4}

__We now create a dictionary of all the food items' variables keeping the following things in mind: -__

* The lower bound is equal to zero
* The category should be continuous ie the decision variables could take continuous values.

__Such adjustment is necessary to enable the non-negative conditions as anyway negative quantities of food is not possible right? Imagine -1 block of tofu - would not make any sense at all! It also ensures that the values are real valued__

In [8]:
# A dictionary called 'food_vars' is created to contain the referenced Variables
food_vars = LpVariable.dicts("Food",food,lowBound=0,cat='Continuous')

In [9]:
food_vars

{'Frozen Broccoli': Food_Frozen_Broccoli,
 'Frozen Corn': Food_Frozen_Corn,
 'Raw Lettuce Iceberg': Food_Raw_Lettuce_Iceberg,
 ' Baked Potatoes': Food__Baked_Potatoes,
 'Tofu': Food_Tofu,
 'Roasted Chicken': Food_Roasted_Chicken,
 'Spaghetti W/ Sauce': Food_Spaghetti_W__Sauce,
 'Raw Apple': Food_Raw_Apple,
 'Banana': Food_Banana,
 'Wheat Bread': Food_Wheat_Bread,
 'White Bread': Food_White_Bread,
 'Oatmeal Cookies': Food_Oatmeal_Cookies,
 'Apple Pie': Food_Apple_Pie,
 'Scrambled Eggs': Food_Scrambled_Eggs,
 'Turkey Bologna': Food_Turkey_Bologna,
 'Beef Frankfurter': Food_Beef_Frankfurter,
 'Chocolate Chip Cookies': Food_Chocolate_Chip_Cookies}

## Addition of Objective fucntion to the LPP. Note the use of the `lpSum method.`

In [10]:
prob += lpSum([costs[i]*food_vars[i] for i in food])
prob

Diet_Problem:
MINIMIZE
0.48*Food_Apple_Pie + 0.44999999999999996*Food_Banana + 0.81*Food_Beef_Frankfurter + 0.09*Food_Chocolate_Chip_Cookies + 0.48*Food_Frozen_Broccoli + 0.54*Food_Frozen_Corn + 0.27*Food_Oatmeal_Cookies + 0.72*Food_Raw_Apple + 0.06*Food_Raw_Lettuce_Iceberg + 2.52*Food_Roasted_Chicken + 0.33*Food_Scrambled_Eggs + 2.34*Food_Spaghetti_W__Sauce + 0.9299999999999999*Food_Tofu + 0.44999999999999996*Food_Turkey_Bologna + 0.15000000000000002*Food_Wheat_Bread + 0.18*Food_White_Bread + 0.18*Food__Baked_Potatoes + 0.0
VARIABLES
Food_Apple_Pie Continuous
Food_Banana Continuous
Food_Beef_Frankfurter Continuous
Food_Chocolate_Chip_Cookies Continuous
Food_Frozen_Broccoli Continuous
Food_Frozen_Corn Continuous
Food_Oatmeal_Cookies Continuous
Food_Raw_Apple Continuous
Food_Raw_Lettuce_Iceberg Continuous
Food_Roasted_Chicken Continuous
Food_Scrambled_Eggs Continuous
Food_Spaghetti_W__Sauce Continuous
Food_Tofu Continuous
Food_Turkey_Bologna Continuous
Food_Wheat_Bread Continuous
Food_W

## Addition of the Constraints

##### Note that we further put our constraints now based upon the maximum and the minimum intake of the nutritional components in our data set. Do not forget the motive, we intend to minimize the cost considering these constraints on the components or the decision variables.

##### For the sake of simplicity, and in order to maintain brevity, I am planning to define only five constraints.
##### Now the lpSum method helps in calculating the sum of the linear expressions, so we will use it to define the constraint of Calories in the data.

In [11]:
lpSum([food_vars[i]*calories[i] for i in food])

67.2*Food_Apple_Pie + 104.9*Food_Banana + 141.8*Food_Beef_Frankfurter + 78.1*Food_Chocolate_Chip_Cookies + 73.8*Food_Frozen_Broccoli + 72.2*Food_Frozen_Corn + 81.0*Food_Oatmeal_Cookies + 81.4*Food_Raw_Apple + 2.6*Food_Raw_Lettuce_Iceberg + 277.4*Food_Roasted_Chicken + 99.6*Food_Scrambled_Eggs + 358.2*Food_Spaghetti_W__Sauce + 88.2*Food_Tofu + 56.4*Food_Turkey_Bologna + 65.0*Food_Wheat_Bread + 65.0*Food_White_Bread + 171.5*Food__Baked_Potatoes + 0.0

In [12]:
df[['Foods','Calories']]

,Foods,Calories
0,Frozen Broccoli,73.8
1,Frozen Corn,72.2
2,Raw Lettuce Iceberg,2.6
3,Baked Potatoes,171.5
4,Tofu,88.2
5,Roasted Chicken,277.4
6,Spaghetti W/ Sauce,358.2
7,Raw Apple,81.4
8,Banana,104.9
9,Wheat Bread,65.0


#### Hence in the two output images above, we could compare that lpSum helps in taking the sumproduct of the food items with their respective size of the nutritional component taken under consideration (Calories in this case)

#### Note that we still have not defined the constraints. We define it by adding the same to our problem statement (Recall the objective function and the constraints are the part of the LPP)

In [13]:
prob += lpSum([food_vars[x]*calories[x] for x in food]) >= 800, "CaloriesMinimum"
prob += lpSum([food_vars[x]*calories[x] for x in food]) <= 1300, "CaloriesMaximum"
# prob

In [14]:
prob

Diet_Problem:
MINIMIZE
0.48*Food_Apple_Pie + 0.44999999999999996*Food_Banana + 0.81*Food_Beef_Frankfurter + 0.09*Food_Chocolate_Chip_Cookies + 0.48*Food_Frozen_Broccoli + 0.54*Food_Frozen_Corn + 0.27*Food_Oatmeal_Cookies + 0.72*Food_Raw_Apple + 0.06*Food_Raw_Lettuce_Iceberg + 2.52*Food_Roasted_Chicken + 0.33*Food_Scrambled_Eggs + 2.34*Food_Spaghetti_W__Sauce + 0.9299999999999999*Food_Tofu + 0.44999999999999996*Food_Turkey_Bologna + 0.15000000000000002*Food_Wheat_Bread + 0.18*Food_White_Bread + 0.18*Food__Baked_Potatoes + 0.0
SUBJECT TO
CaloriesMinimum: 67.2 Food_Apple_Pie + 104.9 Food_Banana
 + 141.8 Food_Beef_Frankfurter + 78.1 Food_Chocolate_Chip_Cookies
 + 73.8 Food_Frozen_Broccoli + 72.2 Food_Frozen_Corn + 81 Food_Oatmeal_Cookies
 + 81.4 Food_Raw_Apple + 2.6 Food_Raw_Lettuce_Iceberg
 + 277.4 Food_Roasted_Chicken + 99.6 Food_Scrambled_Eggs
 + 358.2 Food_Spaghetti_W__Sauce + 88.2 Food_Tofu + 56.4 Food_Turkey_Bologna
 + 65 Food_Wheat_Bread + 65 Food_White_Bread + 171.5 Food__Baked_Pot

__Hence, the output shows how beautifully the function works. We have the objective function  which are subject to the "Calories" constraint as defined in the code above. Now since one of the five nutritional components has been duly defined, let us go ahead and define the remaining four components as well in order to formulate the problem outrightly.__

#### The five nutritional components that I have chosen are `Carbohydrates, Fat, Protein, Vitamin A & Calcium.` ####

In [15]:
#Carbohydrates' constraint
prob += lpSum([food_vars[x]*carbs[x] for x in food]) >= 130, "CarbsMinimum"
prob += lpSum([food_vars[x]*carbs[x] for x in food]) <= 200, "CarbsMaximum"

#Fat's constraint
prob += lpSum([food_vars[x]*fat[x] for x in food]) >= 20, "FatsMinimum"
prob += lpSum([food_vars[x]*fat[x] for x in food]) <= 50, "FatsMaximum"

#Protein's constraint
prob += lpSum([food_vars[x]*protein[x] for x in food]) >= 100, "ProteinsMinimum"
prob += lpSum([food_vars[x]*protein[x] for x in food]) <= 150, "ProteinsMaximum"

#Vit_A constraint
prob += lpSum([food_vars[x]*vit_A[x] for x in food]) >= 1000, "Vit_A_Minimum"
prob += lpSum([food_vars[x]*vit_A[x] for x in food]) <= 10000, "Vit_A_Maximum"

### Running the Solver Algorithm

###### Finally, we have our problem ready and now we shall run the Solver algorithm. Kindly note that we could pass our own parameters while running the solver n terms of the algorithm which could be used, but in this case, I shall run the solver without any parameters and let it decide the best algorithm to run according to the structure of the problem. Yes, this library is quite optimized to do so!

In [16]:
prob.solve()

1

In [17]:
prob.solver

###### Now we print the status of the problem. Note that if the problem is not formulated well then it might have an "infeasible" soltuin or if it does not provide sufficient information then it might be "Unbounded". But our solution is "optimal" which means that the solution is optimized.

In [18]:
LpStatus[prob.status]

'Optimal'

### Printing of the variables

In [19]:
# prob.variables()[1].varValue()

for var in prob.variables():
    print(f'Variable name: {var.name} , Variable value : {var.value()}\n')

print('\n')
print('*'*100)
print('\n')

#We can also see the slack variables of the constraints
for name, con in prob.constraints.items():
    print(f'constraint name:{name}, constraint value:{con.value()}\n')

print('*'*100)
print('\n')

## OBJECTIVE VALUE
print(f'OBJECTIVE VALUE IS: {round(prob.objective.value(),2)}')

Variable name: Food_Apple_Pie , Variable value : 0.0

Variable name: Food_Banana , Variable value : 0.0

Variable name: Food_Beef_Frankfurter , Variable value : 0.0

Variable name: Food_Chocolate_Chip_Cookies , Variable value : 0.0

Variable name: Food_Frozen_Broccoli , Variable value : 1.4079473

Variable name: Food_Frozen_Corn , Variable value : 0.0

Variable name: Food_Oatmeal_Cookies , Variable value : 0.0

Variable name: Food_Raw_Apple , Variable value : 0.0

Variable name: Food_Raw_Lettuce_Iceberg , Variable value : 0.0

Variable name: Food_Roasted_Chicken , Variable value : 1.2329408

Variable name: Food_Scrambled_Eggs , Variable value : 4.0165704

Variable name: Food_Spaghetti_W__Sauce , Variable value : 0.0

Variable name: Food_Tofu , Variable value : 0.0

Variable name: Food_Turkey_Bologna , Variable value : 0.0

Variable name: Food_Wheat_Bread , Variable value : 0.0

Variable name: Food_White_Bread , Variable value : 0.0

Variable name: Food__Baked_Potatoes , Variable value 

### The optimal value of the objective function is $5.58. 

#### The interpretation of the result would be the following: -

* 2.64 servings of baked potatoes

* 4.02 servings of  scrambled eggs

* 1.23 servings of Roasted chicken 

* 1.41 servings of Frozen broccoli

___The following diet plan would give the minimum cost and the maximum nutritional value to the customer. Hence, we have charted out the best food plan while also minimizing the cost.___

##### Since we did not mention any parameter hence the PuLP uses the default solver CBC for the resolution. We could have used a different solver for our own solution. 